In [1]:
from __future__ import division
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import deque
from helper import getData
from helper import create_model
from helper import imageToVertor
from helper import codeToVertor
from define import codeLength

In [ ]:
for i, index in enumerate(range(codeLength), 1):
    model_path = 'model/%s/' % index
    model_file_name = os.path.join(model_path, 'model')
    nodes_file_name = os.path.join(model_path, 'nodes.pk')
    if not os.path.exists(nodes_file_name):
        create_model(model_path)

    stat_length = 30
    recent_accuracy = deque(maxlen=stat_length)
    graph = tf.Graph()
    config = tf.ConfigProto(intra_op_parallelism_threads=2)
    session = tf.Session(graph=graph, config=config)
    with session.graph.as_default():
        # 导入模型定义
        saver = tf.train.import_meta_graph(model_file_name + '.meta')
        saver.restore(session, model_file_name)
        nodes = pickle.load(open(nodes_file_name, "rU"))
        x = session.graph.get_tensor_by_name(nodes['x'])
        y = session.graph.get_tensor_by_name(nodes['y'])
        keep_prob = session.graph.get_tensor_by_name(nodes['keep_prob'])
        loss = session.graph.get_tensor_by_name(nodes['loss'])
        accuracy = session.graph.get_tensor_by_name(nodes['accuracy'])
        optimizer = session.graph.get_operation_by_name(nodes['optimizer'])

        # 训练模型
        for step in range(100000):
            imageList, codeList = getData(100)
            codeList = map(lambda x: x[index], codeList)
            x_data = map(imageToVertor, imageList)
            y_data = map(codeToVertor, codeList)
            _, l, a = session.run(
                [optimizer, loss, accuracy], 
                feed_dict={x: x_data, y: y_data, keep_prob: .75})
            if step % 10 == 0:
                saver.save(session, model_file_name)
            recent_accuracy.append(a)
            mean_of_accuracy = pd.Series(recent_accuracy).mean()
            format_string = '[%d(%d/%d):%d]: loss: %f, accuracy: %f, accuracy mean: %f'
            print format_string % (index, i, codeLength, step, l, a, mean_of_accuracy)
            if len(recent_accuracy) == stat_length:
                if mean_of_accuracy >= .90:
                    break

INFO:tensorflow:Restoring parameters from model/0/model
[0(1/4):0]: loss: 0.085606, accuracy: 0.830000, accuracy mean: 0.830000
[0(1/4):1]: loss: 0.071040, accuracy: 0.860000, accuracy mean: 0.845000
[0(1/4):2]: loss: 0.051783, accuracy: 0.940000, accuracy mean: 0.876667
[0(1/4):3]: loss: 0.078656, accuracy: 0.830000, accuracy mean: 0.865000
[0(1/4):4]: loss: 0.071318, accuracy: 0.900000, accuracy mean: 0.872000
[0(1/4):5]: loss: 0.067236, accuracy: 0.860000, accuracy mean: 0.870000
[0(1/4):6]: loss: 0.060358, accuracy: 0.900000, accuracy mean: 0.874286
[0(1/4):7]: loss: 0.110076, accuracy: 0.780000, accuracy mean: 0.862500
[0(1/4):8]: loss: 0.075075, accuracy: 0.890000, accuracy mean: 0.865556
[0(1/4):9]: loss: 0.071675, accuracy: 0.840000, accuracy mean: 0.863000
[0(1/4):10]: loss: 0.052960, accuracy: 0.900000, accuracy mean: 0.866364
[0(1/4):11]: loss: 0.059965, accuracy: 0.890000, accuracy mean: 0.868333
[0(1/4):12]: loss: 0.081915, accuracy: 0.800000, accuracy mean: 0.863077
[0(1/